# Importando as bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Leitura do dataset

In [4]:
df = pd.read_csv("fraud_oracle.csv")


In [6]:
pd.set_option('display.max_columns', None)

df

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47,Policy Holder,Sport - Collision,Sport,more than 69000,0,3,7,400,3,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65,Third Party,Sedan - Liability,Sport,20000 to 29000,0,4,4,400,2,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,more than 69000,0,5,3,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15415,Nov,4,Friday,Toyota,Urban,Tuesday,Nov,5,Male,Married,35,Policy Holder,Sedan - Collision,Sedan,20000 to 29000,1,15416,5,400,4,more than 30,more than 30,2 to 4,6 years,31 to 35,No,No,External,none,no change,1 vehicle,1996,Collision
15416,Nov,5,Thursday,Pontiac,Urban,Friday,Dec,1,Male,Married,30,Policy Holder,Sedan - Liability,Sport,30000 to 39000,0,15417,11,400,3,more than 30,more than 30,more than 4,6 years,31 to 35,No,No,External,more than 5,no change,3 to 4,1996,Liability
15417,Nov,5,Thursday,Toyota,Rural,Friday,Dec,1,Male,Single,24,Policy Holder,Sedan - Collision,Sedan,20000 to 29000,1,15418,4,400,4,more than 30,more than 30,more than 4,5 years,26 to 30,No,No,External,1 to 2,no change,1 vehicle,1996,Collision
15418,Dec,1,Monday,Toyota,Urban,Thursday,Dec,2,Female,Married,34,Third Party,Sedan - All Perils,Sedan,20000 to 29000,0,15419,6,400,4,more than 30,more than 30,none,2 years,31 to 35,No,No,External,more than 5,no change,1 vehicle,1996,All Perils


# **Explicação das Colunas**

**Informações temporais**
* Month - Mês em que o acidente ocorreu                        
* WeekOfMonth - Semana do mês do acidente                               
* DayOfWeek - Dia da semana do acidente
* DayOfWeekClaimed - Dia da semana em que o sinistro foi registrado.
* MonthClaimed - Mês em que o cliente entrou com a reclamação do seguro.
* WeekOfMonthClaimed - Semana do mês em que a reclamação foi registrada. 
* Year - Ano do registro do caso.

**Informações sobre o veículo**
* Make - Fabricante do carro (Honda, Toyota, etc.)   
* NumberOfCars - Número de carros que o segurado possui.
* AgeOfVehicle - Idade do veículo (ex.: “3 years”, “more than 7”).   
* VehicleCategory - Categoria do veículo (ex.: Sedan, SUV, Sports car, Utility, Van)
* VehiclePrice - Preço/valor do veículo       
     
**Informações sobre o acidente**
* AccidentArea -  Local do acidente (Urban ou Rural).  
* PoliceReportFiled - Indica se boletim policial foi registrado (Yes/No).
* WitnessPresent - Se havia testemunha presente (Yes/No).
* Fault - Culpado pelo acidente (ex.: Policy Holder, Third Party, Unknown, Not At Fault).

**Informações sobre o segurado**
* Sex - Sexo do segurado (Male/Female).
* MaritalStatus - Estado civil (Single, Married, etc.).
* Age - Idade do segurado.
* AgeOfPolicyHolder - Faixa etária do segurado (ex.: “26 to 30”).
* AgentType - Tipo do agente de atendimento (Internal/External).

**Histórico de apólice e comportamento**
* Days_Policy_Accident - Tempo entre emissão da apólice e o acidente
* Days_Policy_Claim - Tempo entre emissão da apólice e abertura da reclamação.
* PastNumberOfClaims - Número de sinistros anteriores do segurado.
* NumberOfSuppliments - Número de reavaliações adicionais no processo de sinistro.
* AddressChange_Claim - Tempo desde a última mudança de endereço.
* BasePolicy -  Tipo de cobertura fundamental que acompanha a apólice (Liability, Collision, etc.).
* PolicyType - Tipo de apólice (ex.: Private, Commercial, Corporate, Special)
* PolicyNumber -  Número único da apólice / identificador do cliente.

**Valores e características do processo**
* RepNumber - Identificação do representante ou agente.
* Deductible - Franquia que o cliente deve pagar.
* DriverRating - Nota de risco do motorista (1–4, geralmente).
* FraudFound_P -  Indicador de fraude encontrada/presumida para aquele sinistro 





# Análise Exploratória

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

### **Quantidade de Linhas e Colunas no DataSet**

In [9]:
print(f'O dataset possui {df.shape[0]} de linhas')
print(f'O dataset possui {df.shape[1]} de colunas')

O dataset possui 15420 de linhas
O dataset possui 33 de colunas


### Verificando se existem dados nulos

In [ ]:

df.isnull().sum()